# DISCLAIMER

This notebook is based on this notebook: 
[VQGAN+Clip (Updated)](https://colab.research.google.com/github/justinjohn0306/VQGAN-CLIP/blob/main/VQGAN%2BCLIP(Updated).ipynb) [![Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/justinjohn0306/VQGAN-CLIP/blob/main/VQGAN%2BCLIP(Updated).ipynb)

As can be seen in the notes from the original notebook below, there were many authors. If you look at the code, you can tell, that there were edits made, that make it hard to understand what the code is doing (global variables, etc.). I did neither fix this nor make it better. I believe I made it a lot worse, from a coding perspective. BUT, the code stands as it is, as a piece of art and for documenting how the images for the FantasyArts projects have been created. That was its purpose and that it will always be ;-) 

## Hints on usage
Just execute all the cells, to create a small sample of random images. Use the checkboxes and fields to modify the script.

# Notes from the original Notebook
## Generate images from text phrases with VQGAN and CLIP (z + quantize method with augmentations).
Notebook by Katherine Crowson (https://github.com/crowsonkb, https://twitter.com/RiversHaveWings). The original BigGAN + CLIP method was made by https://twitter.com/advadnoun. Translated and added explanations, and modifications by Eleiber # 8347, and the friendly interface was made thanks to Abulafia # 3734.

For a detailed tutorial on how to use it, I recommend [visit this article] (https://yourcriatures.miraheze.org/wiki/Help:Create_images_with_VQGAN+CLIP), made by Jakeukalane # 2767 and Avengium (Angel) # 3715

Modified by: Justin John

In [ ]:
#@markdown #**Licensed under the MIT License (*Double-click me to read the license agreement*)**
#@markdown ---

# Copyright (c) 2021 Katherine Crowson

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

In [ ]:
#@markdown V100 = Excellent (*Available only for Colab Pro users*)

#@markdown P100 = Very Good

#@markdown T4 = Good (*preferred*)

#@markdown K80 = Meh

#@markdown P4 = (*Not Recommended*) 

#@markdown ---

!nvidia-smi -L

In [ ]:
mount_gdrive = False #@param {type:"boolean"}
if mount_gdrive:
  from google.colab import drive
  drive.mount('/content/drive')

In [ ]:
#@markdown #**Installation of libraries**
# @markdown This cell will take a little while because it has to download several libraries

#@markdown ---
 
print("Installing CLIP...")
!git clone https://github.com/openai/CLIP                 &> /dev/null
 
print("Installing Python Libraries for AI...")
!git clone https://github.com/CompVis/taming-transformers &> /dev/null
!pip install transformers                                 &> /dev/null
!pip install ftfy regex tqdm omegaconf pytorch-lightning  &> /dev/null
!pip install kornia                                       &> /dev/null
!pip install einops                                       &> /dev/null
!pip install wget                                         &> /dev/null
 
print("Installing libraries for metadata management...")
!pip install stegano                                      &> /dev/null
!apt install exempi                                       &> /dev/null
!pip install python-xmp-toolkit                           &> /dev/null
!pip install imgtag                                       &> /dev/null
!pip install pillow==7.1.2                                &> /dev/null
 
print("Installing Python libraries for creating videos...")
!pip install imageio-ffmpeg                               &> /dev/null
!mkdir steps
print("Installation completed.")

In [ ]:
#@markdown #**Selection of models to download**

#@markdown ##Before you can copy the models from your google drive, you need to 
#@markdown ##download them first and upload them to your google drive. If you 
#@markdown ##are not planning on using this notebook excessively, just use the 
#@markdown ##download_models checkbox. If you plan on using it many times, 
#@markdown ##download_models once, upload them to your google drive, and then just use 
#@markdown ##the copy_models option to download them from your google drive to 
#@markdown ##this colab space.

#@markdown ---

#@markdown By default, the notebook downloads Model 16384 from ImageNet. There are others that are not downloaded by default, since it would be in vain if you are not going to use them, so if you want to use them, simply select the models to download.

#@markdown ---

copy_models = False #@param {type:"boolean"}
download_models = True #@param {type:"boolean"}
upload_models = False #@param {type:"boolean"}

#@markdown ---

imagenet_1024 = False #@param {type:"boolean"}
imagenet_16384 = True #@param {type:"boolean"}
gumbel_8192 = False #@param {type:"boolean"}
coco = False #@param {type:"boolean"}
faceshq = False #@param {type:"boolean"}
wikiart_1024 = False #@param {type:"boolean"}
wikiart_16384 = False #@param {type:"boolean"}
sflckr = False #@param {type:"boolean"}
ade20k = False #@param {type:"boolean"}
ffhq = False #@param {type:"boolean"}
celebahq = False #@param {type:"boolean"}

if imagenet_1024:
  if copy_models: 
    !cp -v drive/MyDrive/vqgan-model/vqgan_imagenet_f16_1024.* .
  if download_models: 
    !curl -L -o vqgan_imagenet_f16_1024.yaml -C - 'https://heibox.uni-heidelberg.de/d/8088892a516d4e3baf92/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1' #ImageNet 1024
  if download_models: 
    !curl -L -o vqgan_imagenet_f16_1024.ckpt -C - 'https://heibox.uni-heidelberg.de/d/8088892a516d4e3baf92/files/?p=%2Fckpts%2Flast.ckpt&dl=1'  #ImageNet 1024
  if copy_models: 
    !cp -v *.yaml drive/MyDrive/vqgan-model/
  if copy_models: 
    !cp -v  *.ckpt drive/MyDrive/vqgan-model/ 
if imagenet_16384:
  if copy_models: 
    !cp -v  drive/MyDrive/vqgan-model/vqgan_imagenet_f16_16384.* .
  if download_models: 
    !curl -L -o vqgan_imagenet_f16_16384.yaml -C - 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1' #ImageNet 16384
  if download_models: 
    !curl -L -o vqgan_imagenet_f16_16384.ckpt -C - 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fckpts%2Flast.ckpt&dl=1' #ImageNet 16384
  if upload_models: 
    !cp -v  *.yaml drive/MyDrive/vqgan-model/
  if upload_models: 
    !cp -v  *.ckpt drive/MyDrive/vqgan-model/ 
if gumbel_8192:
  if copy_models: 
    !cp -v  drive/MyDrive/vqgan-model/gumbel_8192.* .
  if download_models: 
    !curl -L -o gumbel_8192.yaml -C - 'https://heibox.uni-heidelberg.de/d/2e5662443a6b4307b470/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1' #Gumbel 8192
  if download_models: 
    !curl -L -o gumbel_8192.ckpt -C - 'https://heibox.uni-heidelberg.de/d/2e5662443a6b4307b470/files/?p=%2Fckpts%2Flast.ckpt&dl=1' #Gumbel 8192
  if upload_models: 
    !cp -v  *.yaml drive/MyDrive/vqgan-model/
  if upload_models: 
    !cp -v  *.ckpt drive/MyDrive/vqgan-model/ 
if coco:
  if copy_models: 
    !cp -v  drive/MyDrive/vqgan-model/coco.* .
  if download_models: 
    !curl -L -o coco.yaml -C - 'https://dl.nmkd.de/ai/clip/coco/coco.yaml' #COCO
  if download_models: 
    !curl -L -o coco.ckpt -C - 'https://dl.nmkd.de/ai/clip/coco/coco.ckpt' #COCO
  if upload_models: 
    !cp -v  *.yaml drive/MyDrive/vqgan-model/
  if upload_models: 
    !cp -v  *.ckpt drive/MyDrive/vqgan-model/ 
if faceshq:
  if copy_models: 
    !cp -v  drive/MyDrive/vqgan-model/faceshq.* .
  if download_models: 
    !curl -L -o faceshq.yaml -C - 'https://drive.google.com/uc?export=download&id=1fHwGx_hnBtC8nsq7hesJvs-Klv-P0gzT' #FacesHQ
  if download_models: 
    !curl -L -o faceshq.ckpt -C - 'https://app.koofr.net/content/links/a04deec9-0c59-4673-8b37-3d696fe63a5d/files/get/last.ckpt?path=%2F2020-11-13T21-41-45_faceshq_transformer%2Fcheckpoints%2Flast.ckpt' #FacesHQ
  if upload_models: 
    !cp -v  *.yaml drive/MyDrive/vqgan-model/
  if upload_models: 
    !cp -v  *.ckpt drive/MyDrive/vqgan-model/ 
if wikiart_1024: 
  if copy_models: 
    !cp -v  drive/MyDrive/vqgan-model/wikiart_1024.* .
  if download_models: 
    !curl -L -o wikiart_1024.yaml -C - 'http://mirror.io.community/blob/vqgan/wikiart.yaml' #WikiArt 1024
  if download_models: 
    !curl -L -o wikiart_1024.ckpt -C - 'http://mirror.io.community/blob/vqgan/wikiart.ckpt' #WikiArt 1024
  if upload_models: 
    !cp -v  *.yaml drive/MyDrive/vqgan-model/
  if upload_models: 
    !cp -v  *.ckpt drive/MyDrive/vqgan-model/ 
if wikiart_16384: 
  if copy_models: 
    !cp -v  drive/MyDrive/vqgan-model/wikiart_16384.* .
  if download_models: 
    !curl -L -o wikiart_16384.yaml -C - 'http://eaidata.bmk.sh/data/Wikiart_16384/wikiart_f16_16384_8145600.yaml' #WikiArt 16384
  if download_models: 
    !curl -L -o wikiart_16384.ckpt -C - 'http://eaidata.bmk.sh/data/Wikiart_16384/wikiart_f16_16384_8145600.ckpt' #WikiArt 16384
  if upload_models: 
    !cp -v  *.yaml drive/MyDrive/vqgan-model/
  if upload_models: 
    !cp -v  *.ckpt drive/MyDrive/vqgan-model/ 
if sflckr:
  if copy_models: 
    !cp -v  drive/MyDrive/vqgan-model/sflckr.* .
  if download_models: 
    !curl -L -o sflckr.yaml -C - 'https://heibox.uni-heidelberg.de/d/73487ab6e5314cb5adba/files/?p=%2Fconfigs%2F2020-11-09T13-31-51-project.yaml&dl=1' #S-FLCKR
  if download_models: 
    !curl -L -o sflckr.ckpt -C - 'https://heibox.uni-heidelberg.de/d/73487ab6e5314cb5adba/files/?p=%2Fcheckpoints%2Flast.ckpt&dl=1' #S-FLCKR
  if upload_models: 
    !cp -v  *.yaml drive/MyDrive/vqgan-model/
  if upload_models: 
    !cp -v  *.ckpt drive/MyDrive/vqgan-model/ 
if ade20k:
  if copy_models: 
    !cp -v  drive/MyDrive/vqgan-model/ade20k.* .
  if download_models: 
    !curl -L -o ade20k.yaml -C - 'https://static.miraheze.org/intercriaturaswiki/b/bf/Ade20k.txt' #ADE20K
  if download_models: 
    !curl -L -o ade20k.ckpt -C - 'https://app.koofr.net/content/links/0f65c2cd-7102-4550-a2bd-07fd383aac9e/files/get/last.ckpt?path=%2F2020-11-20T21-45-44_ade20k_transformer%2Fcheckpoints%2Flast.ckpt' #ADE20K
  if upload_models: 
    !cp -v  *.yaml drive/MyDrive/vqgan-model/
  if upload_models: 
    !cp -v  *.ckpt drive/MyDrive/vqgan-model/ 
if ffhq:
  if copy_models: 
    !cp -v  drive/MyDrive/vqgan-model/ffhq.* .
  if download_models: 
    !curl -L -o ffhq.yaml -C - 'https://app.koofr.net/content/links/0fc005bf-3dca-4079-9d40-cdf38d42cd7a/files/get/2021-04-23T18-19-01-project.yaml?path=%2F2021-04-23T18-19-01_ffhq_transformer%2Fconfigs%2F2021-04-23T18-19-01-project.yaml&force' #FFHQ
  if download_models: 
    !curl -L -o ffhq.ckpt -C - 'https://app.koofr.net/content/links/0fc005bf-3dca-4079-9d40-cdf38d42cd7a/files/get/last.ckpt?path=%2F2021-04-23T18-19-01_ffhq_transformer%2Fcheckpoints%2Flast.ckpt&force' #FFHQ
  if upload_models: 
    !cp -v  *.yaml drive/MyDrive/vqgan-model/
  if upload_models: 
    !cp -v  *.ckpt drive/MyDrive/vqgan-model/ 
if celebahq:
  if copy_models: 
    !cp -v  drive/MyDrive/vqgan-model/celebahq.* .
  if download_models: 
    !curl -L -o celebahq.yaml -C - 'https://app.koofr.net/content/links/6dddf083-40c8-470a-9360-a9dab2a94e96/files/get/2021-04-23T18-11-19-project.yaml?path=%2F2021-04-23T18-11-19_celebahq_transformer%2Fconfigs%2F2021-04-23T18-11-19-project.yaml&force' #CelebA-HQ
  if download_models: 
    !curl -L -o celebahq.ckpt -C - 'https://app.koofr.net/content/links/6dddf083-40c8-470a-9360-a9dab2a94e96/files/get/last.ckpt?path=%2F2021-04-23T18-11-19_celebahq_transformer%2Fcheckpoints%2Flast.ckpt&force' #CelebA-HQ
  if upload_models: 
    !cp -v  *.yaml drive/MyDrive/vqgan-model/
  if upload_models: 
    !cp -v  *.ckpt drive/MyDrive/vqgan-model/ 

In [ ]:
# @title Loading libraries and definitions {display-mode: "form"}
 
import argparse
import math
from pathlib import Path
import sys
 
sys.path.append('./taming-transformers')
from IPython import display
from base64 import b64encode
from omegaconf import OmegaConf
from PIL import Image
from taming.models import cond_transformer, vqgan
import torch
from torch import nn, optim
from torch.nn import functional as F
from torchvision import transforms
from torchvision.transforms import functional as TF
from tqdm.notebook import tqdm
 
from CLIP import clip
import kornia.augmentation as K
import numpy as np
import imageio
from PIL import ImageFile, Image
from imgtag import ImgTag    # metadatos 
from libxmp import *         # metadatos
import libxmp                # metadatos
from stegano import lsb
import json
ImageFile.LOAD_TRUNCATED_IMAGES = True
 
def sinc(x):
    return torch.where(x != 0, torch.sin(math.pi * x) / (math.pi * x), x.new_ones([]))
 
 
def lanczos(x, a):
    cond = torch.logical_and(-a < x, x < a)
    out = torch.where(cond, sinc(x) * sinc(x/a), x.new_zeros([]))
    return out / out.sum()
 
 
def ramp(ratio, width):
    n = math.ceil(width / ratio + 1)
    out = torch.empty([n])
    cur = 0
    for i in range(out.shape[0]):
        out[i] = cur
        cur += ratio
    return torch.cat([-out[1:].flip([0]), out])[1:-1]
 
 
def resample(input, size, align_corners=True):
    n, c, h, w = input.shape
    dh, dw = size
 
    input = input.view([n * c, 1, h, w])
 
    if dh < h:
        kernel_h = lanczos(ramp(dh / h, 2), 2).to(input.device, input.dtype)
        pad_h = (kernel_h.shape[0] - 1) // 2
        input = F.pad(input, (0, 0, pad_h, pad_h), 'reflect')
        input = F.conv2d(input, kernel_h[None, None, :, None])
 
    if dw < w:
        kernel_w = lanczos(ramp(dw / w, 2), 2).to(input.device, input.dtype)
        pad_w = (kernel_w.shape[0] - 1) // 2
        input = F.pad(input, (pad_w, pad_w, 0, 0), 'reflect')
        input = F.conv2d(input, kernel_w[None, None, None, :])
 
    input = input.view([n, c, h, w])
    return F.interpolate(input, size, mode='bicubic', align_corners=align_corners)
 
 
class ReplaceGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x_forward, x_backward):
        ctx.shape = x_backward.shape
        return x_forward
 
    @staticmethod
    def backward(ctx, grad_in):
        return None, grad_in.sum_to_size(ctx.shape)
 
 
replace_grad = ReplaceGrad.apply
 
 
class ClampWithGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, min, max):
        ctx.min = min
        ctx.max = max
        ctx.save_for_backward(input)
        return input.clamp(min, max)
 
    @staticmethod
    def backward(ctx, grad_in):
        input, = ctx.saved_tensors
        return grad_in * (grad_in * (input - input.clamp(ctx.min, ctx.max)) >= 0), None, None
 
 
clamp_with_grad = ClampWithGrad.apply
 
 
def vector_quantize(x, codebook):
    d = x.pow(2).sum(dim=-1, keepdim=True) + codebook.pow(2).sum(dim=1) - 2 * x @ codebook.T
    indices = d.argmin(-1)
    x_q = F.one_hot(indices, codebook.shape[0]).to(d.dtype) @ codebook
    return replace_grad(x_q, x)
 
 
class Prompt(nn.Module):
    def __init__(self, embed, weight=1., stop=float('-inf')):
        super().__init__()
        self.register_buffer('embed', embed)
        self.register_buffer('weight', torch.as_tensor(weight))
        self.register_buffer('stop', torch.as_tensor(stop))
 
    def forward(self, input):
        input_normed = F.normalize(input.unsqueeze(1), dim=2)
        embed_normed = F.normalize(self.embed.unsqueeze(0), dim=2)
        dists = input_normed.sub(embed_normed).norm(dim=2).div(2).arcsin().pow(2).mul(2)
        dists = dists * self.weight.sign()
        return self.weight.abs() * replace_grad(dists, torch.maximum(dists, self.stop)).mean()
 
 
def parse_prompt(prompt):
    vals = prompt.rsplit(':', 2)
    vals = vals + ['', '1', '-inf'][len(vals):]
    return vals[0], float(vals[1]), float(vals[2])
 
 
class MakeCutouts(nn.Module):
    def __init__(self, cut_size, cutn, cut_pow=1.):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.cut_pow = cut_pow
        self.augs = nn.Sequential(
            K.RandomHorizontalFlip(p=0.5),
            # K.RandomSolarize(0.01, 0.01, p=0.7),
            K.RandomSharpness(0.3,p=0.4),
            K.RandomAffine(degrees=30, translate=0.1, p=0.8, padding_mode='border'),
            K.RandomPerspective(0.2,p=0.4),
            K.ColorJitter(hue=0.01, saturation=0.01, p=0.7))
        self.noise_fac = 0.1
 
 
    def forward(self, input):
        sideY, sideX = input.shape[2:4]
        max_size = min(sideX, sideY)
        min_size = min(sideX, sideY, self.cut_size)
        cutouts = []
        for _ in range(self.cutn):
            size = int(torch.rand([])**self.cut_pow * (max_size - min_size) + min_size)
            offsetx = torch.randint(0, sideX - size + 1, ())
            offsety = torch.randint(0, sideY - size + 1, ())
            cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]
            cutouts.append(resample(cutout, (self.cut_size, self.cut_size)))
        batch = self.augs(torch.cat(cutouts, dim=0))
        if self.noise_fac:
            facs = batch.new_empty([self.cutn, 1, 1, 1]).uniform_(0, self.noise_fac)
            batch = batch + facs * torch.randn_like(batch)
        return batch
 
 
def load_vqgan_model(config_path, checkpoint_path):
    config = OmegaConf.load(config_path)
    if config.model.target == 'taming.models.vqgan.VQModel':
        model = vqgan.VQModel(**config.model.params)
        model.eval().requires_grad_(False)
        model.init_from_ckpt(checkpoint_path)
    elif config.model.target == 'taming.models.cond_transformer.Net2NetTransformer':
        parent_model = cond_transformer.Net2NetTransformer(**config.model.params)
        parent_model.eval().requires_grad_(False)
        parent_model.init_from_ckpt(checkpoint_path)
        model = parent_model.first_stage_model
    elif config.model.target == 'taming.models.vqgan.GumbelVQ':
        model = vqgan.GumbelVQ(**config.model.params)
        print(config.model.params)
        model.eval().requires_grad_(False)
        model.init_from_ckpt(checkpoint_path)
    else:
        raise ValueError(f'unknown model type: {config.model.target}')
    del model.loss
    return model
 
 
def resize_image(image, out_size):
    ratio = image.size[0] / image.size[1]
    area = min(image.size[0] * image.size[1], out_size[0] * out_size[1])
    size = round((area * ratio)**0.5), round((area / ratio)**0.5)
    return image.resize(size, Image.LANCZOS)

def download_img(img_url):
    try:
        return wget.download(img_url,out="input.jpg")
    except:
        return


In [ ]:
#@markdown #**Big bad production function**
#@markdown ---

#@markdown to avoid lots of output steps, keep  the images_interval close to max_iterations

#@markdown ---

i = 0
input_texts = ""
input_model = ""
input_iterations = 0
init_image_path = ""
make_gif = False
def produce_and_download(texts, width = 300, height = 300, model = "vqgan_imagenet_f16_16384", images_interval = 20, init_image = "", target_images="", seed = 1337, max_iterations = 200, input_images = ""):
    global input_texts
    global input_model
    global input_iterations
    global i
    global init_image_path
    global make_gif
    i = 0
    input_texts = texts
    input_model = model
    input_iterations = max_iterations
    init_image_path = init_image
    #atmd #**Parameters**
    #atmd ---
    #texts = bd + "|" + sl + "|" +wd + "|" +it + "|" +pw #"" #atprm {type:"string"}
    #texts = sx + "|" + cl + "|" + bd + "|" + sl + "|" +wd + "|" +it + "|" +pw #"" #atprm {type:"string"}
    #width =  300#atprm {type:"number"}
    #height =  300#atprm {type:"number"}
    #model = "vqgan_imagenet_f16_16384" #atprm ["vqgan_imagenet_f16_16384", "vqgan_imagenet_f16_1024", "wikiart_1024", "wikiart_16384", "coco", "faceshq", "sflckr", "ade20k", "ffhq", "celebahq", "gumbel_8192"]
    #images_interval =  20#atprm {type:"number"}
    #init_image = ""#atprm {type:"string"}
    #target_images = ""#atprm {type:"string"}
    #seed = 1337#atprm {type:"number"}
    #max_iterations = 60#atprm {type:"number"}
    #input_images = ""

    model_names={"vqgan_imagenet_f16_16384": 'ImageNet 16384',"vqgan_imagenet_f16_1024":"ImageNet 1024",
                     "wikiart_1024":"WikiArt 1024", "wikiart_16384":"WikiArt 16384", "coco":"COCO-Stuff", "faceshq":"FacesHQ", "sflckr":"S-FLCKR", "ade20k":"ADE20K", "ffhq":"FFHQ", "celebahq":"CelebA-HQ", "gumbel_8192": "Gumbel 8192"}
    name_model = model_names[model]

    if model == "gumbel_8192":
        is_gumbel = True
    else:
        is_gumbel = False

    if seed == -1:
        seed = None
    if init_image == "None":
        init_image = None
    elif init_image and init_image.lower().startswith("http"):
        init_image = download_img(init_image)


    if target_images == "None" or not target_images:
        target_images = []
    else:
        target_images = target_images.split("|")
        target_images = [image.strip() for image in target_images]

    if init_image or target_images != []:
        input_images = True

    texts = [frase.strip() for frase in texts.split("|")]
    if texts == ['']:
        texts = []


    args = argparse.Namespace(
        prompts=texts,
        image_prompts=target_images,
        noise_prompt_seeds=[],
        noise_prompt_weights=[],
        size=[width, height],
        init_image=init_image,
        init_weight=0.,
        clip_model='ViT-B/32',
        vqgan_config=f'{model}.yaml',
        vqgan_checkpoint=f'{model}.ckpt',
        step_size=0.1,
        cutn=64,
        cut_pow=1.,
        display_freq=images_interval,
        seed=seed,
    )

    #atmd #**Fire up the AI**

    #atmd ---


    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('Using device:', device)
    if texts:
        print('Using texts:', texts)
    if target_images:
        print('Using image prompts:', target_images)
    if args.seed is None:
        seed = torch.seed()
    else:
        seed = args.seed
    torch.manual_seed(seed)
    print('Using seed:', seed)

    model = load_vqgan_model(args.vqgan_config, args.vqgan_checkpoint).to(device)
    perceptor = clip.load(args.clip_model, jit=False)[0].eval().requires_grad_(False).to(device)

    cut_size = perceptor.visual.input_resolution
    if is_gumbel:
        e_dim = model.quantize.embedding_dim
    else:
        e_dim = model.quantize.e_dim

    f = 2**(model.decoder.num_resolutions - 1)
    make_cutouts = MakeCutouts(cut_size, args.cutn, cut_pow=args.cut_pow)
    if is_gumbel:
        n_toks = model.quantize.n_embed
    else:
        n_toks = model.quantize.n_e

    toksX, toksY = args.size[0] // f, args.size[1] // f
    sideX, sideY = toksX * f, toksY * f
    if is_gumbel:
        z_min = model.quantize.embed.weight.min(dim=0).values[None, :, None, None]
        z_max = model.quantize.embed.weight.max(dim=0).values[None, :, None, None]
    else:
        z_min = model.quantize.embedding.weight.min(dim=0).values[None, :, None, None]
        z_max = model.quantize.embedding.weight.max(dim=0).values[None, :, None, None]

    if args.init_image:
        pil_image = Image.open(args.init_image).convert('RGB')
        pil_image = pil_image.resize((sideX, sideY), Image.LANCZOS)
        z, *_ = model.encode(TF.to_tensor(pil_image).to(device).unsqueeze(0) * 2 - 1)
    else:
        one_hot = F.one_hot(torch.randint(n_toks, [toksY * toksX], device=device), n_toks).float()
        if is_gumbel:
            z = one_hot @ model.quantize.embed.weight
        else:
            z = one_hot @ model.quantize.embedding.weight
        z = z.view([-1, toksY, toksX, e_dim]).permute(0, 3, 1, 2)
    z_orig = z.clone()
    z.requires_grad_(True)
    opt = optim.Adam([z], lr=args.step_size)

    normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                     std=[0.26862954, 0.26130258, 0.27577711])

    pMs = []

    for prompt in args.prompts:
        txt, weight, stop = parse_prompt(prompt)
        embed = perceptor.encode_text(clip.tokenize(txt).to(device)).float()
        pMs.append(Prompt(embed, weight, stop).to(device))

    for prompt in args.image_prompts:
        path, weight, stop = parse_prompt(prompt)
        img = resize_image(Image.open(path).convert('RGB'), (sideX, sideY))
        batch = make_cutouts(TF.to_tensor(img).unsqueeze(0).to(device))
        embed = perceptor.encode_image(normalize(batch)).float()
        pMs.append(Prompt(embed, weight, stop).to(device))

    for seed, weight in zip(args.noise_prompt_seeds, args.noise_prompt_weights):
        gen = torch.Generator().manual_seed(seed)
        embed = torch.empty([1, perceptor.visual.output_dim]).normal_(generator=gen)
        pMs.append(Prompt(embed, weight).to(device))

    def synth(z):
        if is_gumbel:
            z_q = vector_quantize(z.movedim(1, 3), model.quantize.embed.weight).movedim(3, 1)
        else:
            z_q = vector_quantize(z.movedim(1, 3), model.quantize.embedding.weight).movedim(3, 1)

        return clamp_with_grad(model.decode(z_q).add(1).div(2), 0, 1)

    def add_xmp_data(nombrefichero):
        imagen = ImgTag(filename=nombrefichero)
        imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'creator', 'VQGAN+CLIP', {"prop_array_is_ordered":True, "prop_value_is_array":True})
        if args.prompts:
            imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'title', " | ".join(args.prompts), {"prop_array_is_ordered":True, "prop_value_is_array":True})
        else:
            imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'title', 'None', {"prop_array_is_ordered":True, "prop_value_is_array":True})
        imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'i', str(i), {"prop_array_is_ordered":True, "prop_value_is_array":True})
        imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'model', name_model, {"prop_array_is_ordered":True, "prop_value_is_array":True})
        imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'seed',str(seed) , {"prop_array_is_ordered":True, "prop_value_is_array":True})
        imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'input_images',str(input_images) , {"prop_array_is_ordered":True, "prop_value_is_array":True})
        #for frases in args.prompts:
        #    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'Prompt' ,frases, {"prop_array_is_ordered":True, "prop_value_is_array":True})
        imagen.close()

    def add_stegano_data(filename):
        data = {
            "title": " | ".join(args.prompts) if args.prompts else None,
            "notebook": "VQGAN+CLIP",
            "i": i,
            "model": name_model,
            "seed": str(seed),
            "input_images": input_images
        }
        lsb.hide(filename, json.dumps(data)).save(filename)

    @torch.no_grad()
    def checkin(i, losses):
        losses_str = ', '.join(f'{loss.item():g}' for loss in losses)
        tqdm.write(f'i: {i}, loss: {sum(losses).item():g}, losses: {losses_str}')
        out = synth(z)
        TF.to_pil_image(out[0].cpu()).save('progress.png')
        add_stegano_data('progress.png')
        add_xmp_data('progress.png')
        display.display(display.Image('progress.png'))

    def ascend_txt():
        global i
        out = synth(z)
        iii = perceptor.encode_image(normalize(make_cutouts(out))).float()

        result = []

        if args.init_weight:
            result.append(F.mse_loss(z, z_orig) * args.init_weight / 2)

        for prompt in pMs:
            result.append(prompt(iii))
        img = np.array(out.mul(255).clamp(0, 255)[0].cpu().detach().numpy().astype(np.uint8))[:,:,:]
        img = np.transpose(img, (1, 2, 0))
        filename = f"steps/{i:04}.png"
        imageio.imwrite(filename, np.array(img))
        add_stegano_data(filename)
        add_xmp_data(filename)
        return result

    def train(i):
        opt.zero_grad()
        lossAll = ascend_txt()
        if i % args.display_freq == 0 and args.display_freq is not -1:
            checkin(i, lossAll)
        loss = sum(lossAll)
        loss.backward()
        opt.step()
        with torch.no_grad():
            z.copy_(z.maximum(z_min).minimum(z_max))

    i = 0
    try:
        with tqdm() as pbar:
            while True:
                train(i)
                if i == max_iterations:
                    break
                i += 1
                pbar.update()
    except KeyboardInterrupt:
        pass


    #atmd #**Download the result video**
    from google.colab import files
    import os

    filename = input_texts.replace("|","-")


    if make_gif:
      init_frame = 50 #This is the frame where the video will start
      last_frame = input_iterations #You can change i to the number of the last frame you want to generate. It will raise an error if that number of frames does not exist.

      min_fps = 1
      max_fps = 24

      total_frames = last_frame-init_frame

      length = 15 #Desired video time in seconds

      frames = []
      tqdm.write('Generating video...')
      for i in range(init_frame,last_frame): #
          filename = f"steps/{i:04}.png"
          frames.append(Image.open(filename))

      #fps = last_frame/10
      fps = np.clip(total_frames/length,min_fps,max_fps)

      from subprocess import Popen, PIPE
      p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-', '-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '17', '-preset', 'veryslow', 'video.mp4'], stdin=PIPE)
      for im in tqdm(frames):
          im.save(p.stdin, 'PNG')
      p.stdin.close()

      print("The video is now being compressed, wait...")
      p.wait()
      print("The video is ready")
      vid_name = os.path.basename(init_image_path) + f"_{input_model}_{filename}_{input_iterations:04}.mp4"
      os.rename(f"video.mp4", vid_name)
      !cp -v "$vid_name" drive/MyDrive/vqgan-output/
    new_name = "steps/" + os.path.basename(init_image_path) + f"_{input_model}_{filename}_{input_iterations:04}.png"
    os.rename(f"steps/{input_iterations:04}.png", new_name)
    !cp -v "$new_name" drive/MyDrive/vqgan-output/


    %rm steps/*.png
    %rm steps/*.gif
    %rm steps/*.mp4
    %rm *.png
    %rm *.gif
    %rm *.mp4

    # for card in [input_iterations]: #,int(input_iterations / 5)): #range(input_iterations):
    #     new_name = "steps/" + os.path.basename(init_image_path) + f"_{input_model}_{filename}_{card:04}.png"
    #     os.rename(f"steps/{card:04}.png", new_name)
    #     !cp -v "$new_name" drive/MyDrive/vqgan-output/
    #     #files.download(new_name)
    # import glob
    # from PIL import Image

    # # filepaths
    # fp_in = "steps/0*.png"
    # fp_out = "steps/" + os.path.basename(init_image_path) + f"_{input_model}_{filename}_{card:04}.gif"

    # # https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#gif
    # img, *imgs = [Image.open(f) for f in sorted(glob.glob(fp_in))]
    # img.save(fp=fp_out, format='GIF', append_images=imgs,
    #         save_all=True, duration=200, loop=0)
    # !cp -v "$fp_out" drive/MyDrive/vqgan-output/



In [ ]:
#@markdown #**Random Production**
#@markdown ---

perform_random_production = True #@param {type:"boolean"}

if perform_random_production:

  network = []

  if imagenet_16384 == True :
      network.append("vqgan_imagenet_f16_16384")

  if wikiart_16384 == True: 
      network.append("wikiart_16384")

  if faceshq == True:    
      network.append("faceshq")

  if sflckr == True:
      network.append("sflckr")

  if ade20k == True:
      network.append("ade20k")

  if ffhq == True:
      network.append("ffhq")

  if celebahq == True:
      network.append("celebahq")

  if gumbel_8192 == True:
      network.append("gumbel_8192")

  if coco == True:
      network.append("coco")

  scene_counter = 0

  scenes = []

  entities = {"angel":0, 
              "reaper":0, 
              "devil":0, 
              "guardian":0, 
              "keeper":0,  
              "devil":0, 
              "god":0,  
              "spirit":0,  
              "mushroom":0,  
              "toad":0, 
              "queen":0, 
              "king":0, 
              "prince":0,  
              "princess":0,  
              "troll":0,  
              "ghost":0,  
              "unicorn":0,  
              "hunter":0,  "hunter":0,  
              "wizard":0,  "wizard":0,  
              "witch":0,  "witch":0,  
              "archer":0,  "archer":0,  
              "samurai":0,  "samurai":0,  
              "knight":0,  "knight":0,  
              "spider":0,  "spider":0,  
              "owl":0,  "owl":0,  
              "wolf":0,  "wolf":0,  
              "snake":0,  "snake":0, 
              "lizard":0,  "lizard":0, 
              "lion":0,  "lion":0, 
              "dragon":0,  "dragon":0, 
              "maker":0,
              "hunter":0,
              "wizard":0,
              "witch":0,
              "archer":0,
              "samurai":0,
              "knight":0,
              "spider":0,
              "owl":0,
              "wolf":0,
              "snake":0,
              "lizard":0,
              "lion":0,
              "dragon":0,
              "maker":0,
    }

  adjectives = {"lovely":0,  
                "creepy":0,  
                "cute":0,  
                "serene":0,  
                "peaceful":0,  
                "beautiful":0,  
                "horror":0,  
                "frozen":0,  "frozen":0,  "frozen":0, 
                "burning":0,  "burning":0,  
                "rotten":0,  "rotten":0, 
                "lightray":0, 
                "golden":0,  "golden":0,
                "frozen":0,
                "burning":0,
                "rotten":0,
                "lightray":0,
                "golden":0,
                "magnificient":0, 
                "calm":0,
                "turbulent":0,
                }

  place_modifiers = {"heaven":0,  
                    "hell":0,  
                    "hellfire":0,  
                    "cloud":0,  
                    "sacred":0,  
                    "underwater":0,  
                    "smoke":0,  "smoke":0,  
                    "mist":0,  "mist":0, 
                    "space":0,  "space":0, 
                    "secret":0, 
                    "hidden":0,
                    "smoke":0,
                    "mist":0,
                    "space":0,
                    "secret":0,
                    "hidden":0,
                    }

  realms = {"science":0,  
            "peace":0,  
            "love":0,  
            "quiet":0,  
            "war":0,  
            "thunder":0,  
            "lightning":0,  
            "rainbow":0,  
            "trash":0, 
            "glass":0,
            "wood":0, 
            "iron":0, 
            "sand":0, 
            "hate":0, 
            "fitness":0, 
            "art":0, 
            "sports":0, 
            "terror":0, 
            "cyber":0, 
            "cyberpunk":0, 
            "steam":0, 
            "steampunk":0, 
            "punk":0, 
            "rock":0, 
            "techno":0,
            "glass":0,
            "wood":0,
            "iron":0,
            "hate":0,
            "fitness":0,
            "art":0,
            "sports":0,
            "terror":0,
            "cyber":0,
            "cyberpunk":0,
            "steam":0,
            "steampunk":0,
            "punk":0,
            "rock":0,
            "techno":0,
            "crypto":0,  
    }

  places = {"hole":0,  
            "pit":0,  
            "cave":0,  
            "castle":0,  
            "library":0,  
            "cathedral":0,  
            "church":0,  
            "dome":0,  
            "stadium":0,  
            "arena":0,  
            "forest":0,  
            "sand dune":0,  
            "grassland":0,  
            "jungle":0,  
            "vulcano":0, 
            "island":0, 
            "beach":0, 
            "river":0, 
            "sea":0,
            "mountain":0, 
            "valley":0, 
            "flatland":0, 
            "city":0, 
            "house":0, 
            "tree":0,
            "sand dune":0,
            "river":0,
            "sea":0,
            "mountain":0,
            "valley":0,
            "flatland":0,
            "city":0,
            "house":0,
            "tree":0,
    }

  num_occurrance = 1#@param {type:"number"}

  image_counter = 0
  reroll_counter = 0
  max_image_counter = 3#@param {type:"number"}

  used_width = 224 #@param {type:"number"}
  used_height =  112#@param {type:"number"}

  used_model = "vqgan_imagenet_f16_16384" #@param {type:"string"}


  #@markdown leave this at -1 to omit images in the notebook output.
  #@markdown set between 1 and 100 to observe image creation, but leave at -1 when 
  #@markdown producing lots of images to avoid crashing your instance.
  used_images_interval =  10#@param {type:"number"}
  used_init_image = "" #@param {type:"string"}
  used_target_images="" #@param {type:"string"}
  #@markdown pick your own seed!
  used_seed = 123456 #@param {type:"number"}
  used_max_iterations =  224#@param {type:"number"}
  used_input_images = "" #@param {type:"string"}

  trick = ". zbrush central" #@param {type:"string"}

  import random

  while image_counter < max_image_counter:
    entity = random.choice(list(entities.keys()))
    realm = random.choice(list(realms.keys()))
    adjective = random.choice(list(adjectives.keys()))
    place_modifier = random.choice(list(place_modifiers.keys()))
    place = random.choice(list(places.keys()))

    text = "the " + entity + " of " + realm + " in " + adjective + " " + place_modifier + " " + place + trick
    #print(text)
    if entities[entity] >= num_occurrance or realms[realm] >= num_occurrance or adjectives[adjective] >= num_occurrance or place_modifiers[place_modifier] >= num_occurrance or places[place] >= num_occurrance:
      #print("rerolling")
      reroll_counter += 1
      continue
    entities[entity] += 1 
    realms[realm] += 1 
    adjectives[adjective] += 1 
    place_modifiers[place_modifier] += 1 
    places[place] += 1 
    image_counter += 1
    produce_and_download( texts = text, width = used_width, height = used_height, model = used_model, images_interval = used_images_interval, init_image = used_init_image, target_images=used_target_images, seed = used_seed, max_iterations = used_max_iterations, input_images = used_input_images)
    print(image_counter)


  print(image_counter)
  print(reroll_counter)

In [ ]:
#@markdown this was an early attempt of making the images, based on the stat names.
perform_stat_production = False #@param {type:"boolean"}

if perform_stat_production:

  network = []

  if imagenet_16384 == True :
      network.append("vqgan_imagenet_f16_16384")

  if wikiart_16384 == True: 
      network.append("wikiart_16384")

  if faceshq == True:    
      network.append("faceshq")

  if sflckr == True:
      network.append("sflckr")

  if ade20k == True:
      network.append("ade20k")

  if ffhq == True:
      network.append("ffhq")

  if celebahq == True:
      network.append("celebahq")

  if gumbel_8192 == True:
      network.append("gumbel_8192")

  if coco == True:
      network.append("coco")

  scene_counter = 0

  scenes = []

  stat1 = [
    "wisdom",
    "brave",
    "coward"
  ]

  stat2 = [
    "soul",
    "love",
    "hate",
  ]

  stat3 = [
    "body",
    "strength",
    "speed",
  ]

  stat4 = [
    "power",
    "cute",
    "fear",
  ]

  stat5 = [
    "intelligence",
    "science",
    "magic",
  ]


  used_width = 224 #@param {type:"number"}
  used_height =  112#@param {type:"number"}

  used_model = "vqgan_imagenet_f16_16384" #@param {type:"string"}

  used_images_interval =  -1#@param {type:"number"}
  used_init_image = "" #@param {type:"string"}
  used_target_images="" #@param {type:"string"}
  used_seed = 123456 #@param {type:"number"}
  used_max_iterations =  200#@param {type:"number"}
  used_input_images = "" #@param {type:"string"}


  stat_combination_count = 0

  for s1 in stat1:
    for s2 in stat2:
      for s3 in stat3:
        for s4 in stat4:
          for s5 in stat5:
            text = "rpg character face " + s1 + " " + s2 + " " + s3 + " " + s4 + " " + s5 + ". zbrush rendered portrait 3ds max vray" 
            print(text)
            stat_combination_count += 1
            print(stat_combination_count)
            produce_and_download( 
                texts = text, 
                width = used_width, 
                height = used_height, 
                model = used_model, 
                images_interval = used_images_interval, 
                init_image = used_init_image, 
                target_images=used_target_images, 
                seed = used_seed, 
                max_iterations = used_max_iterations, 
                input_images = used_input_images
                )



In [ ]:
#@markdown this will zip all images in the steps folder and save them in a output.zip file

zip_images = False #@param {type:"boolean"}

!rm steps/0*.png

if zip_images:
    !zip -r output.zip steps

In [ ]:
#@markdown this is supposed to open a download dialog in the browser for the output.zip file.
download = False #@param {type:"boolean"}
#@markdown this copies all images fromt he step folder to the google drive
save_to_drive = False #@param {type:"boolean"}


if download:
    from google.colab import files
    files.download("output.zip")

if save_to_drive:
    !cp steps/*.png drive/MyDrive/vqgan-output/

In [ ]:
#@markdown this can delete all images and zip files
delete_all = False #@param {type:"boolean"}
delete_zip = False #@param {type:"boolean"}

if delete_all:
    %rm steps/*.png
    %rm steps/*.gif
    %rm steps/*.mp4
    %rm *.png
    %rm *.gif
    %rm *.mp4
if delete_zip:
    %rm output.zip